In [ ]:
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import h5py
import os

import keras
import keras.layers as KL
import keras.models as KM
from keras.callbacks import ModelCheckpoint

import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

from data_utils.hdf5_sequence import HDF5Sequence
from get_model import get_model
C
from constants import MODEL_NAME, MODEL_SAVE_PATH, CHECKPOINT_PATH

%load_ext autoreload
%autoreload 2

## Constants

In [13]:
PRETRAINED_MODEL_NAME = 'hierdec-trio_16bar'
config = configs.CONFIG_MAP[PRETRAINED_MODEL_NAME]

DATA_PATH = './data/lmd_clean/'
HDF5_PATH = DATA_PATH + 'lmd_clean_split_z.h5'

# Definitions moved to get_model
# TIMESTEPS = 256
# DIM_MELODY = 90
# DIM_LATENT = 512

In [14]:
config.hparams

HParams([('batch_size', 256), ('beta_rate', 0.0), ('clip_mode', 'global_norm'), ('conditional', True), ('dec_rnn_size', [1024, 1024]), ('decay_rate', 0.9999), ('dropout_keep_prob', 1.0), ('enc_rnn_size', [2048, 2048]), ('free_bits', 256), ('grad_clip', 1.0), ('grad_norm_clip_to_zero', 10000), ('learning_rate', 0.001), ('max_beta', 0.2), ('max_seq_len', 256), ('min_learning_rate', 1e-05), ('residual_decoder', False), ('residual_encoder', False), ('sampling_rate', 0.0), ('sampling_schedule', 'constant'), ('use_cudnn', False), ('z_size', 512)])

## Hyperparameters

In [15]:
batch_size = 16

## Data pipeline

In [16]:
data_file = h5py.File(HDF5_PATH, 'r')

In [17]:
train_seq = HDF5Sequence(data_file, batch_size, 
                         index_path=DATA_PATH + 'train_indices.csv')
val_seq = HDF5Sequence(data_file, batch_size, 
                       index_path=DATA_PATH + 'val_indices.csv')

## Define model: bidirectional RNN

In [22]:
# Model definition moved to get_model.py, constants.py

In [23]:
checkpoint = ModelCheckpoint(CHECKPOINT_PATH, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [24]:
model = get_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 256, 90)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256, 256)          225280    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               395264    
_________________________________________________________________
output (Dense)               (None, 512)               131584    
Total params: 752,128
Trainable params: 752,128
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer=keras.optimizers.rmsprop(lr=3e-5),
              loss='mean_squared_error',
              metrics=['accuracy'])

In [26]:
model.fit_generator(train_seq, steps_per_epoch=len(train_seq),
                    validation_data=val_seq, validation_steps=len(val_seq),
                    max_queue_size=128, workers=32, epochs=20,
                    callbacks=callbacks_list)

Epoch 1/20
4722/5000 [===========================>..] - ETA: 50s - loss: 0.1707 - acc: 0.0289

KeyboardInterrupt: 

In [1]:
model

NameError: name 'model' is not defined